# Programación Dinámica y Estocástica

## Objetivos

* Aprender cuándo utilizar programación dinámica y sus beneficios.
* Entender las diferencias entre programas deterministas y estocásticos.
* Aprender a utilizar Programación Estocástica
* Aprender a crear simulaciones computacionales válidas.

## Programación dinámica

En la década de los 50s Richard Bellman necesitaba financiamiento del gobierno para poder continuar con sus investigaciones, por lo que necesitaba un nombre rimbombante para que no fueran capaz de rechazar su solicitud, por lo que eligió programación dinámica. Las propias palabras de Bellman fueron:

> “[El nombre] Programación Dinámica se escogió para esconder a patrocinadores gubernamentales el hecho que en realidad estaba haciendo Matemáticas. La frase Programación Dinámica es algo que ningún congresista puede oponerse.” - Richard Bellman.

* **Subestructura Óptima**. Una solución global óptima se puede encontrar al combinar soluciones óptimas de subproblemas locales.

* **Problemas empalmados**. Una solución óptima que involucra resolver el mismo problema en varias ocasiones.

Los problemas que puede optimizar son aquellos que tienen una **subestructura óptima**, esto significa que una solución óptima global se puede encontrar al combinar soluciones óptimas de subproblemas locales. ej: Merge Sort.

También nos podemos encontrar con los **problemas empalmados**, los cuales implican resolver el mismo problema en varias ocasiones para dar con una solución óptima. ej: Fibonacci.

Una técnica para obtener una alta velocidad en nuestro programa es la **Memorización**, el cual consiste en guardar cómputos previos y evitar realizarlos nuevamente. Normalmente se utiliza un diccionario, donde las consultas se pueden hacer en $O(1)$, y para ello hacemos un cambio de tiempo por espacio.

## Optimización de Fibonacci

$$F_n = F_{n-1} + F_{n-2}$$

### Implementación recursiva

In [1]:
def fibonacci_recursivo(n: int) -> int:
    if n == 0 or n == 1:
        return 1
    else:
        return fibonacci_recursivo(n-1) + fibonacci_recursivo(n-2)


In [2]:
%%time
fibonacci_recursivo(40)

CPU times: user 58.5 s, sys: 13.6 ms, total: 58.5 s
Wall time: 58.8 s


165580141

### Implementación dinámica

In [3]:
def fibonacci_dinamico(n: int, memo: dict) -> int:
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fibonacci_dinamico(n-1, memo) + fibonacci_dinamico(n-2, memo)
        memo[n] = result
        return result

In [4]:
%%time
fibonacci_dinamico(40, {})

CPU times: user 86 µs, sys: 1 µs, total: 87 µs
Wall time: 90.8 µs


165580141

In [5]:
%%time
from tqdm import tqdm

memo = {}

for i in tqdm(range(100000)):
    fibonacci_dinamico(i, memo)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 77533.58it/s]

CPU times: user 888 ms, sys: 388 ms, total: 1.28 s
Wall time: 1.32 s


In [6]:
print(memo[500])

225591516161936330872512695036072072046011324913758190588638866418474627738686883405015987052796968498626


## Caminos aleatorios

* Es un tipo de simulación que elige aleatoriamente una decisión dentro de un conjunto de decisiones válidas.
* Se utiliza en muchos campos del conocimiento cuando los sistemas no son deterministas e incluyen elementos de aleatoriedad.

### Camino de borrachos

In [7]:
import random


class Borracho:
    def __init__(self, nombre: str):
        self.nombre = nombre


class BorrachoTradicional(Borracho):
    def __init__(self, nombre: str):
        super().__init__(nombre)

    def camina(self) -> tuple:
        return random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])


In [27]:
coordenadas_x = [0]
coordenadas_y = [0]

class Coordenada:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def mover(self, delta_x: int, delta_y: int) -> 'Coordenada':
        return Coordenada(self.x + delta_x, self.y + delta_y)

    def distancia(self, otra_coordenada: 'Coordenada') -> float:
        delta_x = self.x - otra_coordenada.x
        delta_y = self.y - otra_coordenada.y
        return (delta_x**2 + delta_y**2)**0.5


class Campo:
    def __init__(self):
        self.coordenadas = {}

    def anhadir_borracho(self, borracho: Borracho, coordenada: Coordenada):
        self.coordenadas[borracho] = coordenada

    def mover_borracho(self, borracho: Borracho):
        delta_x, delta_y = borracho.camina()
        
        coordenada_actual = self.coordenadas[borracho]
        coordenada_nueva = coordenada_actual.mover(delta_x, delta_y)
        
        coordenadas_x.append(coordenada_nueva.x)
        coordenadas_y.append(coordenada_nueva.y)
        
        self.coordenadas[borracho] = coordenada_nueva

    def obtener_coordenada(self, borracho: Borracho) -> Coordenada:
        return self.coordenadas[borracho]


In [32]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

def caminata(campo: Campo, borracho: Borracho, pasos: int) -> float:
    inicio = campo.obtener_coordenada(borracho)
    for _ in range(pasos):
        campo.mover_borracho(borracho)

    return inicio.distancia(campo.obtener_coordenada(borracho))


def simular_caminata(pasos: int, numero_intentos: int, tipo_de_borracho: type):
    borracho = tipo_de_borracho('Juan')
    origen = Coordenada(0, 0)
    distancias = []

    for _ in range(numero_intentos):
        campo = Campo()
        campo.anhadir_borracho(borracho, origen)
        simulacion_caminata = caminata(campo, borracho, pasos)
        distancias.append(round(simulacion_caminata, 1))

    return distancias

def graficar(x: list, y: list):
    grafica = figure(title='Camino aleatorio', x_axis_label='pasos', y_axis_label='distancia')
    grafica.line(x, y, legend_label='distancia media')

    show(grafica)

def simular(distancias_de_caminatas: list, numero_intentos: int, tipo_de_borracho: type, show_graph: bool = False):
    distancia_media_por_caminata = []
    caminos = {}
    
    for pasos in distancias_de_caminatas:
        distancias = simular_caminata(pasos, numero_intentos, tipo_de_borracho)
        distancia_media = round(sum(distancias)/len(distancias), 4)
        distancia_maxima = max(distancias)
        distancia_minima = min(distancias)

        distancia_media_por_caminata.append(distancia_media)

        print(f'{tipo_de_borracho.__name__} caminata de {pasos} pasos')
        print(f'Media = {distancia_media}')
        print(f'Max = {distancia_maxima}')
        print(f'Min = {distancia_minima}')
        print()
    
    if show_graph:
        graficar(distancias_de_caminatas, distancia_media_por_caminata)

In [33]:
output_notebook()

distancias_de_caminatas = [10, 100, 1000, 10000]

numero_de_intentos = 100

simular(distancias_de_caminatas, numero_de_intentos, BorrachoTradicional, show_graph=True)

Loading BokehJS ...

BorrachoTradicional caminata de 10 pasos
Media = 2.795
Max = 7.6
Min = 0.0

BorrachoTradicional caminata de 100 pasos
Media = 8.886
Max = 22.1
Min = 1.4

BorrachoTradicional caminata de 1000 pasos
Media = 26.759
Max = 70.0
Min = 3.2

BorrachoTradicional caminata de 10000 pasos
Media = 93.347
Max = 195.9
Min = 12.2



In [48]:
coordenadas_x = [0]
coordenadas_y = [0]

simular([10000], 1, BorrachoTradicional)

grafica = figure(title='Caminos aleatorios')
grafica.line(coordenadas_x, coordenadas_y)

show(grafica)

BorrachoTradicional caminata de 10000 pasos
Media = 60.1
Max = 60.1
Min = 60.1

